In [0]:
import tensorflow as tf
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data

In [2]:
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use urllib or similar directly.
Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py fr

In [0]:
def layer(input, weight_shape, bias_shape):
    weight_std = (2/weight_shape[0])**0.5
    weight_init = tf.random_normal_initializer(stddev = weight_std)
    bias_init = tf.constant_initializer(value=0)

    W = tf.get_variable("W", weight_shape, initializer=weight_init)
    b = tf.get_variable("b", bias_shape, initializer=bias_init)

    return tf.nn.relu(tf.matmul(input,W)+b)

In [0]:
def conv2d (input, weight_shape, bias_shape):
    inp = weight_shape[0] * weight_shape[1] * weight_shape[2]
    weight_init = tf.random_normal_initializer(stddev = (2.0/inp)**0.5)
    
    W = tf.get_variable("W", weight_shape, initializer = weight_init)
    
    bias_init = tf.constant_initializer(value=0)
    b = tf.get_variable("b", bias_shape, initializer=bias_init)
    
    conv_out = tf.nn.conv2d(input, W, strides=[1,1,1,1], padding='SAME')
    
    return tf.nn.relu(tf.nn.bias_add(conv_out,b))

In [0]:
def max_pool(input,k=2):
    return tf.nn.max_pool(input, ksize = [1,k,k,1], strides = [1,k,k,1], padding='SAME')

In [0]:
def inference(x, keep_prob):
    
    x = tf.reshape(x, shape=[-1,28,28,1])
    with tf.variable_scope("conv_1"):
        conv_1 = conv2d(x,[5,5,1,32], [32])
        max_pool_1 = max_pool(conv_1)

    with tf.variable_scope("conv_2"):
        conv_2 = conv2d(max_pool_1,[5,5,32,64],[64])
        max_pool_2 = max_pool(conv_2)

    with tf.variable_scope("fc"):
        pool_2_flat = tf.reshape(max_pool_2,[-1, 7*7*64])
        fc1 = layer(pool_2_flat,[7*7*64, 1024],[1024])
        
        fc_dropout = tf.nn.dropout(fc1, keep_prob)
    
    with tf.variable_scope("output"):
        output = layer(fc_dropout, [1024,10], [10])
    
    return output

    return output

In [0]:
# cross entropy loss of the batch
def loss(output,y):

    xentropy = tf.nn.softmax_cross_entropy_with_logits_v2(labels=y,logits=output)
    loss = tf.reduce_mean(xentropy)
    return loss

# optimizer which will reduce the error and update gradients
def training(cost, global_step):
    tf.summary.scalar("cost",cost)

    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    train_op = optimizer.minimize(cost,global_step=global_step)
    return train_op

#evaluation of the model
def evaluation(output,y):
    correct_pred = tf.equal(tf.argmax(output,1),tf.argmax(y,1))
    accuracy = tf.reduce_mean(tf.cast(correct_pred,tf.float32))
    return accuracy


In [0]:
#Parameters 
learning_rate = 0.01
training_epochs = 300
batch_size = 150
display_step =20

In [9]:
with tf.Graph().as_default():

    #mnist data image of 28*28=784
    x = tf.placeholder("float",[None,784])

    #digits recognition placeholder
    y = tf.placeholder("float",[None,10])

    output = inference(x,0.5)
    cost = loss(output,y)
    global_step = tf.Variable(0,name="global_step",trainable=False)

    train_op = training(cost,global_step)
    eval_op = evaluation(output,y)

    summary_op = tf.summary.merge_all()

    saver = tf.train.Saver()
    sess = tf.Session()
    summary_writer = tf.summary.FileWriter("feed_forward_logs/",graph_def = sess.graph_def)

    init_op = tf.initialize_all_variables()

    sess.run(init_op)

    #Training Cycle

    for epoch in range(training_epochs):
        avg_cost = 0
        total_batch = int(mnist.train.num_examples/batch_size)

        #Loop over all batches
        for i in range(total_batch):
            mbatch_x, mbatch_y = mnist.train.next_batch(batch_size)

            feed_dict = {x: mbatch_x, y:mbatch_y}
            sess.run(train_op, feed_dict= feed_dict)

            minibatch_cost = sess.run(cost, feed_dict= feed_dict)
            avg_cost += minibatch_cost/total_batch

            if epoch%display_step == 0 :
                val_feed_dict = {
                    x: mnist.validation.images,
                    y: mnist.validation.labels
                }

                accuracy = sess.run(eval_op, feed_dict=val_feed_dict)
                print( "Validation error: {} ".format(1-accuracy))

                summary_str = sess.run(summary_op,feed_dict=feed_dict)

                summary_writer.add_summary(summary_str,sess.run(global_step))

                saver.save(sess, "feed_forward_logs/model-checkpoint", global_step = global_step)

            if epoch%10 == 0:
                print("Epoch {}".format(epoch))

    print ("Optimization Finished")

    test_feed_dict = {
        x: mnist.test.images,
        y: mnist.test.labels
    }

    accuracy = sess.run(eval_op, feed_dict = test_feed_dict)

    print ("Testing accuracy: {}".format(accuracy))


Instructions for updating:
Use `tf.global_variables_initializer` instead.
Validation error: 0.8856000006198883 
Epoch 0
Validation error: 0.8769999966025352 
Epoch 0
Validation error: 0.8612000048160553 
Epoch 0
Validation error: 0.852400004863739 
Epoch 0
Validation error: 0.8346000015735626 
Epoch 0
Validation error: 0.8291999995708466 
Epoch 0
Validation error: 0.80519999563694 
Epoch 0
Validation error: 0.8025999963283539 
Epoch 0
Validation error: 0.7856000065803528 
Epoch 0
Validation error: 0.777199998497963 
Epoch 0
Validation error: 0.7681999951601028 
Epoch 0
Validation error: 0.756400004029274 
Epoch 0
Validation error: 0.751800000667572 
Epoch 0
Validation error: 0.7314000129699707 
Epoch 0
Validation error: 0.7188000082969666 
Epoch 0
Validation error: 0.7068000137805939 
Epoch 0
Validation error: 0.6962000131607056 
Epoch 0
Validation error: 0.6929999887943268 
Epoch 0
Validation error: 0.6775999963283539 
Epoch 0
Validation error: 0.6563999950885773 
Epoch 0
Validation e

KeyboardInterrupt: ignored